In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


In [2]:
# load dataset
df = pd.read_csv('train_2v.csv')

In [3]:
stroked_count=0
for i in range(0, len(df.index)):
    if df.iloc[i]['stroke'] == 1:
        stroked_count +=1

no_stroked_count=0
for i in range(0, len(df.index)):
    if df.iloc[i]['stroke'] == 0:
        no_stroked_count +=1

print("No Strokes:", no_stroked_count)

print("Strokes:",stroked_count)

print(df)

No Strokes: 42617
Strokes: 783
          id  gender   age  hypertension  heart_disease ever_married  \
0      30669    Male   3.0             0              0           No   
1      30468    Male  58.0             1              0          Yes   
2      16523  Female   8.0             0              0           No   
3      56543  Female  70.0             0              0          Yes   
4      46136    Male  14.0             0              0           No   
5      32257  Female  47.0             0              0          Yes   
6      52800  Female  52.0             0              0          Yes   
7      41413  Female  75.0             0              1          Yes   
8      15266  Female  32.0             0              0          Yes   
9      28674  Female  74.0             1              0          Yes   
10     10460  Female  79.0             0              0          Yes   
11     64908    Male  79.0             0              1          Yes   
12     63884  Female  37.0       

In [4]:
#Data Quality Report Before Preprocessing
# find datatype of each column
data_types = pd.DataFrame(df.dtypes, columns=['Data Type'])

# count number of missing observations by column
missing_data_counts = pd.DataFrame(df.isnull().sum(), columns=['Missing Values'])

# count number of present observations by column
present_data_counts = pd.DataFrame(df.count(), columns=['Present Values'])

# count number of unique observations by column
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(df.columns.values):
	unique_value_counts.loc[v] = [df[v].nunique()]

data_quality_report = pd.concat([present_data_counts, missing_data_counts, unique_value_counts], axis=1)
print(data_quality_report)

                   Present Values  Missing Values Unique Values
id                          43400               0         43400
gender                      43400               0             3
age                         43400               0           104
hypertension                43400               0             2
heart_disease               43400               0             2
ever_married                43400               0             2
work_type                   43400               0             5
Residence_type              43400               0             2
avg_glucose_level           43400               0         12543
bmi                         41938            1462           555
smoking_status              30108           13292             3
stroke                      43400               0             2


In [5]:
#Here we can observe that we have two attributes with missing values i.e bmi and smoking_status

#Since bmi is a numerical variable we find mean it's mean, median and mode

print("For bmi")
print("Mean:", df['bmi'].mean())
print("Median:", df['bmi'].median())
print("Mode:", df['bmi'].mode())

#We observe that mean, median and mode for bmi are nearly the same
#So we replace the missing values with mean
df['bmi']= df['bmi'].replace(np.nan, df['bmi'].mean())
#print(df)

For bmi
Mean: 28.605038390004545
Median: 27.7
Mode: 0    27.7
dtype: float64


In [6]:
no_stroke_data =0
stroke_data =0

df=df.fillna('NA')
for i in range(len(df.index)):
    if df.iloc[i]['smoking_status'] == 'NA' and df.iloc[i]['stroke'] == 0:
        no_stroke_data=no_stroke_data+1
        
print(no_stroke_data)
stroke_data = 13292 - no_stroke_data
print(stroke_data)

#We have an imbalanced dataset with very few data entries which corresponds to "Stroke", so we cannot remove the 
#rows where smoking_status is missing

13147
145


In [7]:
df['Male'] = 0
print(len(df.index))

for i in range(0, len(df.index)):
    if df.iloc[i]['gender'] == 'Male':
        df.loc[i,'Male'] = 1
df = df.drop('gender', axis=1)

df['married'] = 0

for i in range(0, len(df.index)):
	if df.ix[i]['ever_married'] == 'Yes':
		df.loc[i,'married'] = 1
df = df.drop('ever_married', axis=1)
   
df['urban'] = 0

for i in range(0, len(df.index)):
	if df.ix[i]['Residence_type'] == 'Urban':
		df.loc[i,'urban'] = 1
df = df.drop('Residence_type', axis=1)

df['children']=0
df['Private']=0
df['Self-employed']=0
df['Govt_job']=0
df['Never_worked']=0
for i in range(0, len(df.index)):
    if df.ix[i]['work_type'] == 'children':
        df.loc[i,'children'] = 1
    if df.ix[i]['work_type'] == 'Private':
        df.loc[i,'Private'] = 1
    if df.ix[i]['work_type'] == 'Self-employed':
        df.loc[i,'Self-employed'] = 1
    if df.ix[i]['work_type'] == 'Govt_job':
        df.loc[i,'Govt_job'] = 1
    if df.ix[i]['work_type'] == 'Never_worked':
        df.loc[i,'Never_worked'] = 1
    
df = df.drop('work_type', axis=1)


print(df)

43400


C:\Users\Vidhi Bansal\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
C:\Users\Vidhi Bansal\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Vidhi Bansal\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Vidhi Bansal\Anaconda3\lib\site-packages\ipykernel_launcher.p

          id   age  hypertension  heart_disease  avg_glucose_level        bmi  \
0      30669   3.0             0              0              95.12  18.000000   
1      30468  58.0             1              0              87.96  39.200000   
2      16523   8.0             0              0             110.89  17.600000   
3      56543  70.0             0              0              69.04  35.900000   
4      46136  14.0             0              0             161.28  19.100000   
5      32257  47.0             0              0             210.95  50.100000   
6      52800  52.0             0              0              77.59  17.700000   
7      41413  75.0             0              1             243.53  27.000000   
8      15266  32.0             0              0              77.67  32.300000   
9      28674  74.0             1              0             205.84  54.600000   
10     10460  79.0             0              0              77.08  35.000000   
11     64908  79.0          

In [8]:
#Without Smoking Status
df1 = df.drop('smoking_status', axis=1)
print(df1.head(10))
smt = SMOTE()
X_train = df1.drop('stroke', axis=1).values
y_train = df1['stroke'].values
X_train, y_train = smt.fit_sample(X_train, y_train)

      id   age  hypertension  heart_disease  avg_glucose_level   bmi  stroke  \
0  30669   3.0             0              0              95.12  18.0       0   
1  30468  58.0             1              0              87.96  39.2       0   
2  16523   8.0             0              0             110.89  17.6       0   
3  56543  70.0             0              0              69.04  35.9       0   
4  46136  14.0             0              0             161.28  19.1       0   
5  32257  47.0             0              0             210.95  50.1       0   
6  52800  52.0             0              0              77.59  17.7       0   
7  41413  75.0             0              1             243.53  27.0       0   
8  15266  32.0             0              0              77.67  32.3       0   
9  28674  74.0             1              0             205.84  54.6       0   

   Male  married  urban  children  Private  Self-employed  Govt_job  \
0     1        0      0         1        0      

In [9]:
np.bincount(y_train)

#Both categories have equal amount of records
#More specifically, the minority class has been increased to the total number of majority class.

array([42617, 42617], dtype=int64)

In [10]:
#print(df.corr(method='pearson'))
#So out of 783, losing 145 data entries i.e 18% of the stroked data is not a very good choice

#dff = pd.DataFrame(X_train, columns=['id','age','hypertension','heart_disease','avg_glucose_level','bmi','Male','married','urban','children','Private','Self-employed','Govt_job','Never_worked'])
#dff['stroke'] = 0

#dff.to_csv('tt.csv')

In [12]:
#dee = pd.DataFrame(y_train,columns=['stroke'])
#dee.to_csv('final_y.csv')